In [78]:

# Standard mathematical functions
import math  
# Operating system  functions
import os, sys, glob, json, time, copy
from pathlib import Path

import tarfile
import urllib.request

import numpy as np 
import pandas as pd

pd.set_option("display.max_columns", 200)
from fastparquet import ParquetFile
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from mlxtend.plotting import scatterplotmatrix
import seaborn as sns

# Plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff

pd.options.display.max_columns = None
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
from IPython.display import display,HTML
import plotly.express as px

## ML Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from keras.layers import Dense
from tensorflow.keras import layers
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
import keras.backend as K

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler

In [123]:
testml = pd.read_parquet('../data/AnalysisDF4_MLmodels.parquet')
# Drop columns without relevant information or duplicated like WH2[kg/s] is similar to WH2[g/s]
testml = testml.drop(columns = ['A0_Streamtube_m2','D0_Streamtube_m', 'WH2[kg/s]'])
dataset = testml[['Alt[ft]','Mach','RPM',"Tamb",'Pamb[Pa]','SFC[kg/Ns]','FN[N]','WH2[g/s]','Pt3[Pa]','Nozzle_Tt8[K]']]
dataset.describe()

,Alt[ft],Mach,RPM,Tamb,Pamb[Pa],SFC[kg/Ns],FN[N],WH2[g/s],Pt3[Pa],Nozzle_Tt8[K]
count,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000
mean,16400.000000,0.400000,0.860560,255.589050,57953.045353,0.000012,338.992599,3.705640,297504.982199,694.701007
std,10374.890978,0.258264,0.123054,20.598555,23700.593137,0.000005,207.492350,2.112164,141254.868152,112.738121
min,0.000000,0.000000,0.370238,223.028100,26436.233513,0.000008,20.584683,0.433902,40881.858686,439.762979
25%,6560.000000,0.200000,0.798044,236.052480,35599.775931,0.000010,171.723603,2.019397,191519.734525,600.666697
50%,16400.000000,0.400000,0.892349,255.589050,54019.880008,0.000011,323.710583,3.372669,270887.647735,687.639830
75%,26240.000000,0.600000,0.960524,275.125620,79495.197435,0.000012,480.351208,4.997470,385656.691033,803.527439
max,32800.000000,0.800000,1.005003,288.150000,101325.000000,0.000049,950.804945,10.612773,765939.198725,987.818530


In [124]:
dataset.nunique() 

Alt[ft]            11
Mach                9
RPM              1980
Tamb               11
Pamb[Pa]           11
SFC[kg/Ns]       1980
FN[N]            1980
WH2[g/s]         1980
Pt3[Pa]          1980
Nozzle_Tt8[K]    1980
dtype: int64

In [2]:
# In order to fine tune the model we need to wrap our Keras model in objects
# that mimic Scikit-Learn regressors

# get the model
def build_model(n_inputs, n_outputs, hidden_layer_sizes = 2, n_neurons = 128, lr = 1e-3,name = 'test'):
    
    model = keras.Sequential(name = name)
    model.add(keras.layers.InputLayer(input_shape = n_inputs))
    for layer in range(hidden_layer_sizes):
        model.add(keras.layers.Dense(n_neurons, activation = 'relu'))
    # Output Layer
    model.add(Dense(n_outputs))
    optim = keras.optimizers.Adam(learning_rate = lr)
    model.compile(loss='mse', 
                  optimizer= optim, 
                  metrics =[tf.keras.metrics.MeanSquaredError()])
    return model


def evaluate_model(X, y):
    results = []
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = build_model(n_inputs, n_outputs,hidden_layer_sizes = 2, n_neurons = 128, lr = 1e-3,name = 'test')
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        mse_history = model.evaluate(X_test, y_test, verbose=0)
        # store result
        #print(mse_history[0])
        results.append(mse_history[0])
    return results

In [128]:
dataset_num_atr = ["Tamb",'Pamb[Pa]','RPM','SFC[kg/Ns]','FN[N]','WH2[g/s]','Pt3[Pa]','Nozzle_Tt8[K]']
dataset_binned_atr = ['Alt[ft]','Mach']
n_binned_vars = np.arange(0,len(dataset_binned_atr))
n_bins = np.arange(2,8,2)

data = []

for n_vars, var in enumerate(dataset_binned_atr):
    binned_vars = dataset_binned_atr[0:n_vars+1]
    not_binned_vars = dataset_binned_atr[n_vars + 1:] + dataset_num_atr
    for bins in n_bins:
        # Pipeline
        num_pipeline = make_pipeline(StandardScaler())
        column_trans = ColumnTransformer(
            [
                ("binned_numeric", KBinsDiscretizer(n_bins = bins), binned_vars),
                ('numeric', num_pipeline, not_binned_vars),
            ]
        )

        dataset_prepared_trans = column_trans.fit_transform(dataset)
        
        features = dataset_prepared_trans[:,:-4]
        targets = dataset_prepared_trans[:,-4:]
        print(features.shape, targets.shape)
        
        # Train / Valid / Test splits
        X_train_full, X_test, y_train_full, y_test = train_test_split(features, targets, test_size = 0.3, random_state = 0)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size = 0.2, random_state = 0)
        
        # Test Several NN simple configurations
        n_neurons = np.arange(8,72,8)
        n_layers = np.arange(1,4,1)
        for l in n_layers:
            for n in n_neurons:
                model = build_model(X_train_full.shape[1], y_train_full.shape[1],
                                    hidden_layer_sizes = l, n_neurons = n, lr = 1e-3)
        # fit model
                    
                history = model.fit(X_train_full, y_train_full, epochs=100, verbose=0)
                
                print('min loss ' + str(np.min(history.history['loss'])))
                #train_min_mse.append(np.min(history.history['mean_squared_error']))
                print('train min mse ' + str(np.min(history.history['mean_squared_error'])))
                
                np.save('./NN_binning_eval/history_'+'n_vars_bin_'+ str(len(binned_vars)) + '_bins_'+ str(bins) + '_nlayers_' + str(l)+ '_neurons_' + str(n) + '_.npy',history.history)
                
                valid_scores = model.evaluate(X_valid, y_valid)
                
                data.append([len(binned_vars), 
                             bins, 
                             l, 
                             n,
                             np.min(history.history['mean_squared_error']),
                             valid_scores[1], 
                            ])
                #test_mse.append(test_scores[1])
                print('train mse ' + str(np.min(history.history['mean_squared_error'])) + '/' + ' validation mse ' + str(valid_scores[1]))
        
        

(1980, 7) (1980, 4)
min loss 0.013097390532493591
train min mse 0.013097390532493591
9/9 [==============================] - 0s 2ms/step - loss: 0.0154 - mean_squared_error: 0.0154
train mse 0.013097390532493591/ validation mse 0.015410076826810837
min loss 0.004860274493694305
train min mse 0.004860274493694305
9/9 [==============================] - 0s 1ms/step - loss: 0.0065 - mean_squared_error: 0.0065
train mse 0.004860274493694305/ validation mse 0.006454497575759888
min loss 0.005517732352018356
train min mse 0.005517732352018356
9/9 [==============================] - 0s 1ms/step - loss: 0.0083 - mean_squared_error: 0.0083
train mse 0.005517732352018356/ validation mse 0.008302411995828152
min loss 0.0022080340422689915
train min mse 0.0022080340422689915
9/9 [==============================] - 0s 1ms/step - loss: 0.0030 - mean_squared_error: 0.0030
train mse 0.0022080340422689915/ validation mse 0.003021436743438244
min loss 0.0013310657814145088
train min mse 0.001331065781414508

min loss 0.0008634469122625887
train min mse 0.0008634469122625887
9/9 [==============================] - 0s 1ms/step - loss: 7.2487e-04 - mean_squared_error: 7.2487e-04
train mse 0.0008634469122625887/ validation mse 0.0007248672773130238
min loss 0.00046882868628017604
train min mse 0.00046882868628017604
9/9 [==============================] - 0s 1ms/step - loss: 5.0154e-04 - mean_squared_error: 5.0154e-04
train mse 0.00046882868628017604/ validation mse 0.0005015396163798869
min loss 0.0003715981438290328
train min mse 0.0003715981438290328
9/9 [==============================] - 0s 2ms/step - loss: 4.8183e-04 - mean_squared_error: 4.8183e-04
train mse 0.0003715981438290328/ validation mse 0.00048183463513851166
min loss 0.0002883403794839978
train min mse 0.0002883403794839978
9/9 [==============================] - 0s 997us/step - loss: 3.0484e-04 - mean_squared_error: 3.0484e-04
train mse 0.0002883403794839978/ validation mse 0.0003048373619094491
min loss 0.0002354237949475646
tra

9/9 [==============================] - 0s 997us/step - loss: 0.0107 - mean_squared_error: 0.0107
train mse 0.008236760273575783/ validation mse 0.010735896416008472
min loss 0.0038792691193521023
train min mse 0.0038792691193521023
9/9 [==============================] - 0s 997us/step - loss: 0.0046 - mean_squared_error: 0.0046
train mse 0.0038792691193521023/ validation mse 0.004581918474286795
min loss 0.002496614120900631
train min mse 0.002496614120900631
9/9 [==============================] - 0s 997us/step - loss: 0.0026 - mean_squared_error: 0.0026
train mse 0.002496614120900631/ validation mse 0.002619638340547681
min loss 0.0019344318425282836
train min mse 0.0019344318425282836
9/9 [==============================] - 0s 867us/step - loss: 0.0021 - mean_squared_error: 0.0021
train mse 0.0019344318425282836/ validation mse 0.002060502767562866
min loss 0.00161486875731498
train min mse 0.00161486875731498
9/9 [==============================] - 0s 993us/step - loss: 0.0015 - mean_s

In [139]:
report_df = pd.DataFrame(data, columns = ['binned_vars','n_bins','n_layers','n_neurons_per_layer','train_mse','val_mse'])
report_df.head()

,binned_vars,n_bins,n_layers,n_neurons_per_layer,train_mse,val_mse
0,1,2,1,8,0.013097,0.015410
1,1,2,1,16,0.004860,0.006454
2,1,2,1,24,0.005518,0.008302
3,1,2,1,32,0.002208,0.003021
4,1,2,1,40,0.001331,0.002210


In [140]:
report_df.tail()

,binned_vars,n_bins,n_layers,n_neurons_per_layer,train_mse,val_mse
139,2,6,3,32,0.001269,0.001390
140,2,6,3,40,0.000983,0.001360
141,2,6,3,48,0.000997,0.001091
142,2,6,3,56,0.000741,0.000892
143,2,6,3,64,0.000701,0.000907


In [141]:
report_df['Delta_val_train_mse'] = report_df['val_mse'] - report_df['train_mse']
report_df.head()

,binned_vars,n_bins,n_layers,n_neurons_per_layer,train_mse,val_mse,Delta_val_train_mse
0,1,2,1,8,0.013097,0.015410,0.002313
1,1,2,1,16,0.004860,0.006454,0.001594
2,1,2,1,24,0.005518,0.008302,0.002785
3,1,2,1,32,0.002208,0.003021,0.000813
4,1,2,1,40,0.001331,0.002210,0.000879


In [142]:
report_df.to_parquet('../data/Effect_of_Bins_in_NN.parquet',engine="pyarrow")

In [143]:
report_df[['train_mse','val_mse']].plot(kind = 'scatter')

In [151]:
import plotly.graph_objects as go

import numpy as np

x0 = report_df['train_mse']
x1 = report_df['val_mse']


fig = go.Figure()
fig.add_trace(go.Histogram(x=x0,histnorm='probability'))
fig.add_trace(go.Histogram(x=x1,histnorm='probability'))

# The two histograms are drawn on top of another
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.5)
fig.show()

In [153]:
import plotly.figure_factory as ff
import numpy as np

x0 = report_df['train_mse']
x1 = report_df['val_mse']


hist_data = [x0, x1]

group_labels = ['Train MSE', 'Val MSE']
colors = ['#A56CC1', '#A6ACEC']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, colors=colors,
                         bin_size=.2, show_rug=True)

# Add title
fig.update_layout(title_text='Hist and Curve Plot')
fig.show()

In [154]:
report_df['Delta_val_train_mse'].plot()

In [155]:
# Find top 5 models with minimum positive delta

top_min_delta = report_df[(report_df['Delta_val_train_mse'] > 0) & (report_df['Delta_val_train_mse'] < 0.01)]
top_min_delta.head(10)

,binned_vars,n_bins,n_layers,n_neurons_per_layer,train_mse,val_mse,Delta_val_train_mse
0,1,2,1,8,0.013097,0.015410,0.002313
1,1,2,1,16,0.004860,0.006454,0.001594
2,1,2,1,24,0.005518,0.008302,0.002785
3,1,2,1,32,0.002208,0.003021,0.000813
4,1,2,1,40,0.001331,0.002210,0.000879
5,1,2,1,48,0.001699,0.002920,0.001221
6,1,2,1,56,0.001247,0.002383,0.001137
7,1,2,1,64,0.001110,0.001913,0.000803
8,1,2,2,8,0.007481,0.010047,0.002566
9,1,2,2,16,0.002241,0.002871,0.000630


In [156]:
top_min_delta['Delta_val_train_mse'].plot(kind = 'scatter')

In [157]:
top_min_delta = top_min_delta.sort_values(by = 'Delta_val_train_mse')
top_5_min_delta = top_min_delta.iloc[0:6,:]
top_5_min_delta.to_csv('Top_5_NN_confs.csv',header = True)
top_5_min_delta

,binned_vars,n_bins,n_layers,n_neurons_per_layer,train_mse,val_mse,Delta_val_train_mse
89,2,2,3,16,0.002933,0.002939,0.000006
38,1,4,2,56,0.000288,0.000305,0.000016
66,1,6,3,24,0.000883,0.000903,0.000021
44,1,4,3,40,0.000442,0.000465,0.000023
119,2,4,3,64,0.000627,0.000656,0.000030
36,1,4,2,40,0.000469,0.000502,0.000033


In [159]:
# Convert to dict with keys 'binned_vars', 'n_bins', 'n_layers','n_neurons_per_layer'

top_5_min_delta.iloc[0:5,:4].to_dict(orient = 'list')

{'binned_vars': [2, 1, 1, 1, 2],
 'n_bins': [2, 4, 6, 4, 4],
 'n_layers': [3, 2, 3, 3, 3],
 'n_neurons_per_layer': [16, 56, 24, 40, 64]}

24